In [2]:
import psycopg2
from psycopg2 import sql

# List of tables to update
tables = ['kapsiwon_features', 'mokong', 'mokong_features','taito','taito_features']  # Replace with your table names

# Database connection parameters
db_params = {
    'dbname': 'main_db',
    'user': 'postgres',
    'password': 'smug5961',
    'host': 'localhost',
    'port': '5432',
}

def convert_geometries_to_2d(table_name, conn):
    with conn.cursor() as cursor:
        try:
            # Step 1: Create a temporary column to store 2D geometries
            cursor.execute(
                sql.SQL("ALTER TABLE {} ADD COLUMN geom_temp geometry(MultiPolygon, 4326)").format(sql.Identifier(table_name))
            )

            # Step 2: Convert existing geometries to 2D and store them in the temporary column
            cursor.execute(
                sql.SQL("UPDATE {} SET geom_temp = ST_Force2D(geom)").format(sql.Identifier(table_name))
            )

            # Step 3: Drop the original column
            cursor.execute(
                sql.SQL("ALTER TABLE {} DROP COLUMN geom").format(sql.Identifier(table_name))
            )

            # Step 4: Rename the temporary column to the original column name
            cursor.execute(
                sql.SQL("ALTER TABLE {} RENAME COLUMN geom_temp TO geom").format(sql.Identifier(table_name))
            )

            # Step 5: Recreate the spatial index (if necessary)
            # cursor.execute(
              #  sql.SQL("CREATE INDEX IF NOT EXISTS idx_{}_geom ON {} USING gist (geom)").format(sql.Identifier(table_name), sql.Identifier(table_name))
            # )

            # Commit the transaction
            conn.commit()
            print(f"Successfully converted geometries to 2D for table: {table_name}")

        except Exception as e:
            conn.rollback()
            print(f"Failed to convert geometries to 2D for table: {table_name}. Error: {e}")

def main():
    try:
        # Connect to the database
        conn = psycopg2.connect(**db_params)
        for table in tables:
            convert_geometries_to_2d(table, conn)
        conn.close()

    except Exception as e:
        print(f"Error connecting to the database: {e}")

if __name__ == "__main__":
    main()


Successfully converted geometries to 2D for table: kapsiwon_features
Failed to convert geometries to 2D for table: kapsiwon_points. Error: Geometry type (Point) does not match column type (MultiPolygon)

Successfully converted geometries to 2D for table: mokong
Successfully converted geometries to 2D for table: mokong_features
Failed to convert geometries to 2D for table: mokong_points. Error: Geometry type (Point) does not match column type (MultiPolygon)

Successfully converted geometries to 2D for table: taito
Successfully converted geometries to 2D for table: taito_features
Failed to convert geometries to 2D for table: taito_points. Error: Geometry type (Point) does not match column type (MultiPolygon)

